In [1]:
%load_ext kedro.extras.extensions.ipython

[02/07/24 13:06:15] WARNING  /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/e ]8;id=126905;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=293423;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             xtras/extensions/ipython.py:18: DeprecationWarning:                                   
                             kedro.extras.extensions.ipython should be accessed only using the                     
                             alias kedro.ipython. The unaliased name will be removed in Kedro                      
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=735252;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=901351;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/07/24 13:06:15] INFO     Kedro project raftel                                                   ]8;id=566569;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=421983;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=307690;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=14920;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=223890;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=751654;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [2]:
%reload_kedro

                    INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=869538;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=458135;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/07/24 13:06:16] INFO     Kedro project raftel                                                   ]8;id=690195;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=517269;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=524657;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=559276;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=851145;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=954139;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [4]:
master_table = catalog.load("aras_a@spark")

                    INFO     Loading data from 'aras_a@spark' (SparkDataSet)...                 ]8;id=830538;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=50072;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

### Add a Column to numerate the activities order

In [5]:
w = Window.orderBy("SEC")
w2 = Window.orderBy(F.col("ACTIVITY_CHANGED").desc(), "SEC")

master_table = master_table.withColumn(
    "ACTIVITY_CHANGED", 
    F.when(
        (F.col("LABEL") != F.lag("LABEL", 1).over(w)) | (F.lag("LABEL", 1).over(w).isNull()),
        1
    ).otherwise(0)
).withColumn(
    "LR1_CHANGED",
    F.when(
        (F.col("LR1") != F.lag("LR1", 1).over(w)) | (F.lag("LR1", 1).over(w).isNull()),
        1
    ).otherwise(0)
).withColumn(
    "LR2_CHANGED",
    F.when(
        (F.col("LR2") != F.lag("LR2", 1).over(w)) | (F.lag("LR2", 1).over(w).isNull()),
        1
    ).otherwise(0)
).withColumn(
    "ACTIVITY_NUMBER",
    F.when(
        F.col("ACTIVITY_CHANGED") == 1,
        F.row_number().over(w2)
    )
).withColumn(
    "ACTIVITY_NUMBER",
    F.last('ACTIVITY_NUMBER', True).over(w)
)

### Add LR1 y LR2 activity lags (3 activities)

In [6]:
w = Window.orderBy("SEC")

master_table = (
    master_table.withColumn(
        "Lag1_LABEL",
        F.when(
            F.col("ACTIVITY_CHANGED") == 1,
            F.lag("LABEL", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag1_LABEL",
        F.last('Lag1_LABEL', True).over(w)
    ).withColumn(
        "Lag2_LABEL",
        F.when(
            F.col("ACTIVITY_CHANGED") == 1,
            F.lag("Lag1_LABEL", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag2_LABEL",
        F.last('Lag2_LABEL', True).over(w)
    ).withColumn(
        "Lag3_LABEL",
        F.when(
            F.col("ACTIVITY_CHANGED") == 1,
            F.lag("Lag2_LABEL", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag3_LABEL",
        F.last('Lag3_LABEL', True).over(w)
    )
)

In [7]:
master_table = (
    master_table.withColumn(
        "Lag1_LR1",
        F.when(
            F.col("LR1_CHANGED") == 1,
            F.lag("LR1", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag1_LR1",
        F.last('Lag1_LR1', True).over(w)
    ).withColumn(
        "Lag2_LR1",
        F.when(
            F.col("LR1_CHANGED") == 1,
            F.lag("Lag1_LR1", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag2_LR1",
        F.last('Lag2_LR1', True).over(w)
    ).withColumn(
        "Lag3_LR1",
        F.when(
            F.col("LR1_CHANGED") == 1,
            F.lag("Lag2_LR1", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag3_LR1",
        F.last('Lag3_LR1', True).over(w)
    )
)

In [8]:
master_table = (
    master_table.withColumn(
        "Lag1_LR2",
        F.when(
            F.col("LR2_CHANGED") == 1,
            F.lag("LR2", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag1_LR2",
        F.last('Lag1_LR2', True).over(w)
    ).withColumn(
        "Lag2_LR2",
        F.when(
            F.col("LR2_CHANGED") == 1,
            F.lag("Lag1_LR2", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag2_LR2",
        F.last('Lag2_LR2', True).over(w)
    ).withColumn(
        "Lag3_LR2",
        F.when(
            F.col("LR2_CHANGED") == 1,
            F.lag("Lag2_LR2", 1).over(w).cast(IntegerType())
        )
    ).withColumn(
        "Lag3_LR2",
        F.last('Lag3_LR2', True).over(w)
    )
)

In [9]:
master_table = master_table.filter(
    (F.col("Lag1_LABEL").isNotNull()) & 
    (F.col("Lag2_LABEL").isNotNull()) & 
    (F.col("Lag3_LABEL").isNotNull()) &
    (F.col("Lag1_LR1").isNotNull()) & 
    (F.col("Lag2_LR1").isNotNull()) & 
    (F.col("Lag3_LR1").isNotNull()) &
    (F.col("Lag1_LR2").isNotNull()) & 
    (F.col("Lag2_LR2").isNotNull()) & 
    (F.col("Lag3_LR2").isNotNull())
).select(
    'ACTIVITY_NUMBER',
    'DAY',
    'SEC',
    'Ph1',
    'Ph2',
    'Ir1',
    'Fo1',
    'Fo2',
    'Di3',
    'Di4',
    'Ph3',
    'Ph4',
    'Ph5',
    'Ph6',
    'Co1',
    'Co2',
    'Co3',
    'So1',
    'So2',
    'Di1',
    'Di2',
    'Te1',
    'Fo3',
    'Lag3_LR1',
    'Lag2_LR1',
    'Lag1_LR1',
    'LR1',
    'Lag3_LR2',
    'Lag2_LR2',
    'Lag1_LR2',
    'LR2',
    'Lag3_LABEL',
    'Lag2_LABEL',
    'Lag1_LABEL',
    'LABEL',
).orderBy(
    "SEC"
)

In [10]:
catalog.save("master_table@spark", master_table)

[02/07/24 13:06:19] INFO     Saving data to 'master_table@spark' (SparkDataSet)...              ]8;id=541480;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=614143;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#382\382]8;;\

24/02/07 13:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/07 13:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/07 13:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/07 13:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/07 13:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/07 13:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/07 1